In [36]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline

# transfer learning example
# aim to make 10 type judge become 2 type judge, and use the weight already trained
# method : read old weight, lock conv_layer weight 

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Conv2D, MaxPooling2D,Flatten
from keras.optimizers import SGD

from keras.datasets import mnist

from keras.utils import np_utils

# load datas
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# CNN format
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

# extract datas of result euqal to 0, 1 
x_train_01=x_train[y_train<=1]
x_test_01 =x_test[y_test<=1]

y_train_01=y_train[y_train<=1]
y_test_01=y_test[y_test<=1]

# 1-hot encoding
y_train10=np_utils.to_categorical(y_train,10)
y_test10=np_utils.to_categorical(y_test,10)

y_train_01=np_utils.to_categorical(y_train_01,2)
y_test_01=np_utils.to_categorical(y_test_01,2)

#check data is correct 
print('datas : ',
x_train_01.shape,x_test_01.shape,
y_test_01.shape, y_test_01.shape
)

conv_layer = [Conv2D(32,(3,3),padding='same',input_shape=(28,28,1)),
             Activation('relu'),
             MaxPooling2D(pool_size=(2,2)),
             
             Conv2D(64,(3,3),padding='same',input_shape=(28,28,1)),
             Activation('relu'),
             MaxPooling2D(pool_size=(2,2)),
              
              Conv2D(128,(3,3),padding='same',input_shape=(28,28,1)),
             Activation('relu'),
             MaxPooling2D(pool_size=(2,2)),
             ]

fc_layer = [Flatten(),
           Dense(200),
           Activation('relu'),
           Dense(200),
           Activation('softmax')]

model = Sequential(conv_layer+fc_layer)
model.summary()


new_fc_layer =[Flatten(),
              Dense(100),
              Activation('sigmoid'),
              Dense(2),
              Activation('softmax')]

model_0_to_1 = Sequential(conv_layer+new_fc_layer)
model_0_to_1.summary()

for layer in conv_layer:
    layer.trainable=False

print('after lock weight')
model_0_to_1.summary()

model_0_to_1.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])
model_0_to_1.fit(x_train_01,y_train_01,batch_size=100,epochs=5)


env: KERAS_BACKEND=tensorflow
datas :  (12665, 28, 28, 1) (2115, 28, 28, 1) (2115, 2) (2115, 2)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_55 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_56 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 7, 7, 64)          0         
_______________________

In [37]:
score = model_0_to_1.evaluate(x_test_01,y_test_01)

2115/2115 [==============================] - 1s 239us/step


In [39]:
print('loss : ',score[0])
print('accuracy : ',score[1])

loss :  0.001383047850287066
accuracy :  0.9990543723106384
